# Practical Exercises for Formulation Lecture

## Overfitting Example

In this example, a high degree polynomial is able to fit the training data perfectly, but has much worst test error compared to a linear function. The training and test examples are generated with the function $y=x$ with Gaussian noise added.

In [ ]:
# Overfitting example.
# Noisy training and test set generated using a linear function
# Overfit when learning with high degree polynomial

%matplotlib inline  

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np

# Set seed for random number generator to make results reproducible
np.random.seed(1) 

# Number of data points in train and test sets
data_size = 100
data_interval = 10.0/data_size

# Linear regression model
linear = LinearRegression(fit_intercept=True,normalize=True)
# Polynomial regression model, degree the same as data size
poly = Pipeline([('poly', PolynomialFeatures(degree=data_size)),
                 ('lin', LinearRegression(fit_intercept=True, normalize=True))])

# Construct training set
# Odd number between 1 to 10 as training inputs
# Output is y = x + noise
xtrain = np.arange(data_interval/2, 10, data_interval)
train_noise = np.random.normal(0, 1, data_size)
ytrain = xtrain + train_noise

# Fit the models
linear = linear.fit(xtrain[:, np.newaxis], ytrain)
poly = poly.fit(xtrain[:, np.newaxis], ytrain)

# Construct test set
# Even numbers between 1 to 10 as test inputs
xtest = np.arange(data_interval,10 + data_interval/2, data_interval)
test_noise = np.random.normal(0, 1, data_size)

# Do predictions
linear_pred = linear.predict(xtest[:,np.newaxis])
poly_pred = poly.predict(xtest[:,np.newaxis])

# Measure mean squared error
ytest = xtest + test_noise
linerror = mean_squared_error(ytest, linear_pred)
polyerror = mean_squared_error(ytest, poly_pred)

# Plotting
x_plot = np.linspace(0, 10, 100)

fig = plt.figure(1, figsize=(12, 9))
fig.clf()

sub1 = fig.add_subplot(2,2,1)
sub1.set_title('Lin Reg Train Set')
sub1.scatter(xtrain, ytrain,  color='red')
sub1.plot(x_plot, linear.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub2 = fig.add_subplot(2,2,2)
sub2.set_title('Lin Reg Test Set')
sub2.scatter(xtest, ytest,  color='red')
sub2.plot(x_plot, linear.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub3 = fig.add_subplot(2,2,3)
sub3.set_title('Poly Reg Train Set')
sub3.scatter(xtrain, ytrain,  color='red')
sub3.plot(x_plot, poly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub4 = fig.add_subplot(2,2,4)
sub4.set_title('Poly Reg Test Set')
sub4.scatter(xtest, ytest,  color='red')
sub4.plot(x_plot, poly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

fig.show()

In [ ]:
print "Linear test set error: " + "{0:.2f}".format(linerror)
print "Poly test set error: " + "{0:.2f}".format(polyerror)

## Regularization
Regularization is one method for combatting overfitting. Regularized linear regression is often called ridge regression. In the following, we try to reduce overfitting for linear regression using polynomial features by doing ridge regression. In scikit learn, ridge regression finds $\min_w ||Xw - y||_2^2 +\alpha||w||_2^2$.

In [ ]:
from sklearn.linear_model import Ridge

# Polynomial regression model with ridge regression, degree the same as data size
ridgepoly = Pipeline([('poly', PolynomialFeatures(degree=data_size)),
                      ('ridgereg', Ridge(alpha = 0.01, fit_intercept=True,normalize=True))])
ridgepoly = ridgepoly.fit(xtrain[:, np.newaxis], ytrain)
ridgepoly_pred = ridgepoly.predict(xtest[:,np.newaxis])
ridgepolyerror = mean_squared_error(ytest, ridgepoly_pred)

fig = plt.figure(1, figsize=(12, 4.5))
fig.clf()

sub1 = fig.add_subplot(1,2,1)
sub1.set_title('Poly Reg Test Set')
sub1.scatter(xtest, ytest,  color='red')
sub1.plot(x_plot, poly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub2 = fig.add_subplot(1,2,2)
sub2.set_title('Ridge Poly Reg Test Set')
sub2.scatter(xtest, ytest,  color='red')
sub2.plot(x_plot, ridgepoly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

fig.show()

In [ ]:
print "Ridge poly test set error: " + "{0:.2f}".format(ridgepolyerror)

Be careful if the test set is quite different from the training set.

In [ ]:
print "Prediction at x = 15 is " + "{0:.2e}".format(ridgepoly.predict([[15]])[0]) + " when the true value is 15."

# Feature Selection: Sparsity Inducing Regularizer
Regularizing using the $\ell_1$ norm rather than the $\ell_2$ norm induces sparsity and can serve as a method for feature selection. This is often called Lasso. The optimization objective for Lasso is $\frac{1}{2m}||y - Xw||^2_2 + \alpha ||w||_1$.

In [ ]:
from sklearn.linear_model import Lasso

# Polynomial regression model with Lasso, degree the same as data set size
lassopoly = Pipeline([('poly', PolynomialFeatures(degree=data_size)),
                      ('lassoreg', Lasso(alpha = 0.005, fit_intercept=True,normalize=True))])
lassopoly = lassopoly.fit(xtrain[:, np.newaxis], ytrain)
lassopoly_pred = lassopoly.predict(xtest[:,np.newaxis])
lassopolyerror = mean_squared_error(ytest, lassopoly_pred)

fig = plt.figure(1, figsize=(12, 4.5))
fig.clf()

sub1 = fig.add_subplot(1,2,1)
sub1.set_title('Ridge Poly Reg Test Set')
sub1.scatter(xtest, ytest,  color='red')
sub1.plot(x_plot, ridgepoly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub2 = fig.add_subplot(1,2,2)
sub2.set_title('Lasso Poly Reg Test Set')
sub2.scatter(xtest, ytest,  color='red')
sub2.plot(x_plot, lassopoly.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

fig.show()

In [ ]:
print "Lasso poly test set error: " + "{0:.2f}".format(lassopolyerror)
print "Lasso coeffs: " + repr(lassopoly.named_steps['lassoreg'].coef_)
print "Ridge coeffs: " + repr(ridgepoly.named_steps['ridgereg'].coef_)

## Feature Transformation and Normalization
Feature transformation and normalization can affect performance of some learning methods. Consider the same problem learned using ridge regression and lasso without normalization.

In [ ]:
unnormalizedridge = Pipeline([('poly', PolynomialFeatures(degree=data_size)),
                      ('ridgereg', Ridge(alpha = 0.01, fit_intercept=True,normalize=False))])
unnormalizedridge = unnormalizedridge.fit(xtrain[:, np.newaxis], ytrain)
unnormalizedridgepoly_pred = unnormalizedridge.predict(xtest[:,np.newaxis])

unnormalizedlasso = Pipeline([('poly', PolynomialFeatures(degree=data_size)),
                              ('lassoreg',Lasso(alpha = 0.005, fit_intercept=True,
                                                normalize=False))])
unnormalizedlasso = unnormalizedlasso.fit(xtrain[:, np.newaxis], ytrain)
unnormalizedlasso_pred = unnormalizedlasso.predict(xtest[:,np.newaxis])

fig = plt.figure(1, figsize=(12, 4.5))
fig.clf()

sub1 = fig.add_subplot(1,2,1)
sub1.set_title('Unnormalized Ridge Reg Test Set')
sub1.scatter(xtest, ytest,  color='red')
sub1.plot(x_plot, unnormalizedridge.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

sub2 = fig.add_subplot(1,2,2)
sub2.set_title('Unnormalized Lasso Reg Test Set')
sub2.scatter(xtest, ytest,  color='red')
sub2.plot(x_plot, unnormalizedlasso.predict(x_plot[:,np.newaxis]), color='green',linewidth=3)

fig.show()

## Model Selection
In the following example, we put together model selection using cross validation for selecting the regularization parameter for regularized logistic regression and the number of components of PCA to use. 

In [ ]:
# Modified from http://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, decomposition, datasets

logistic = linear_model.LogisticRegression()

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

digits = datasets.load_digits()

# Show the images
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(1, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)
plt.show()

X_digits = digits.data
y_digits = digits.target

###############################################################################
# Plot the PCA spectrum
pca.fit(X_digits)

plt.figure(6, figsize=(4, 3))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_')

###############################################################################
# Prediction

n_components = [20, 40, 64]
Cs = np.logspace(-3, 0, 3)

#Parameters of pipelines can be set using ‘__’ separated parameter names:

estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                              logistic__C=Cs))
estimator.fit(X_digits, y_digits)

plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
plt.legend(prop=dict(size=12))
plt.show()
estimator.cv_results_

## Discriminative vs Generative
In this section, we compare learning with discriminative and generative models. We will use linear discriminant analysis as the generative model and logistic regression as the discriminative model.

For details on scikit learn's
* Linear Discriminant Analysis, see http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.htmlFor 
* Logistic regression, see http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression.

For the first example, we plot the learning curves for data generated from two isotropic Gaussians with centers $(-1, -1, -1, -1), (1, 1, 1, 1)$ and standard deviations of 1. In this case, both models are able to represent the optimal decision boundary and we would like to see which method converges faster.

In [ ]:
# Code modified from http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
from sklearn.datasets import make_blobs
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.05, 1.0, 10)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

n_samples = 1000    
# Generate samples from two Gaussians centers
centers = [(-1, -1, -1, -1), (1, 1, 1, 1)]
X, y = make_blobs(n_samples=n_samples, n_features=4, cluster_std=1.0,
                  centers=centers, shuffle=True, random_state=1)

title = "Learning Curves (LDA)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = LinearDiscriminantAnalysis()
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves (Logistic Regression)"
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = LogisticRegression(C=1000) # large C to avoid regularization
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)

plt.show()

We now look at a handwritten digit classification problem. In this case, the true model is unknown. We regularize to try to get reasonable convergence.

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X, y = digits.data, digits.target

title = "Learning Curves (LDA)"
# Cross validation with 10 iterations
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = LinearDiscriminantAnalysis(solver='eigen', shrinkage=0.4)
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves (Logistic Regression)"
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = LogisticRegression(C=0.005, multi_class='multinomial', solver='newton-cg')
plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)

plt.show()